In [1]:
from __future__ import print_function 
from bs4 import BeautifulSoup
from BeautifulSoup import SoupStrainer as sopstrain
import subprocess
import os
import re
import numpy as np
import string
import operator
import csv
import urllib2
import json
from collections import defaultdict
import StringIO

In [61]:
knowledgGraph = "cnet" # conceptnet5AssocMod, gov, conceptnet5AssocMi, conceptnet5AssocHdl
#knowledgGraph_ = ''.join([k.capitalize() if i>0 else k for i, k in enumerate(knowledgGraph)])
collection = "trec7n8"
method = "lr" # hal, mi, assoc, assocMi, assocHal, assoc2
simMeasure = "" # mi, cnet
projectDir = "/home/fj9124/projects/ir/seq_kb_ir/" 
indexDir = "/scratch/index/indri_5_7/"
colMethodConfigsDir = os.path.join(projectDir, "configs", collection, method)
cfgInFileName = os.path.join(projectDir, "configs", collection, "query" + collection.capitalize()) 
print("cfg infile Name =", cfgInFileName)
cfgOutFileName=os.path.join(colMethodConfigsDir, knowledgGraph, "indriRunQuery.cfg") 
print("cfg outfile Name =", cfgOutFileName)
colIndexDir = os.path.join(indexDir, collection) 
print("colIndexDir =", colIndexDir)
if knowledgGraph in {"cnet"}:
    knowledgGraphIndexDir = os.path.join(indexDir, collection)   
else:
    knowledgGraphIndexDir = os.path.join(indexDir, knowledgGraph)   
print("knowledgGraph index Dir =", knowledgGraphIndexDir)
graphsDir = os.path.join(projectDir, "graphs")
#methodGraphsDir = os.path.join(graphsDir, method)
methodGraphsFileName = []
if method == "hal":
    methodGraphsFileName = [os.path.join(graphsDir, method, knowledgGraph + ".txt")]
    print("methodGraphsFileName =", methodGraphsFileName)
elif method.translate(None, string.digits) in {"mi", "assoc"}:
    methodGraphsFileName = [os.path.join(graphsDir, method.translate(None, string.digits), knowledgGraph, "graph.txt")]
    print("methodGraphsFileName =", methodGraphsFileName)
elif method in {"assocMi"}:
    if knowledgGraph == "conceptnet5AssocGov":
        methodGraphsFileName = [os.path.join(graphsDir, "assoc", collection, "conceptnet5AssocMod" + ".txt")]
        methodGraphsFileName += [os.path.join(graphsDir, "mi", collection, "gov" + ".txt")]
        print("methodGraphsFileName =", methodGraphsFileName)
elif method in {"assocHal"}:
    if knowledgGraph == "conceptnet5AssocGov":
        methodGraphsFileName = [os.path.join(graphsDir, "assoc", collection, "conceptnet5AssocMod" + ".txt")]
        methodGraphsFileName += [os.path.join(graphsDir, "hal", "gov" + ".txt")]
        print("methodGraphsFileName =", methodGraphsFileName)
qrelsDir = os.path.join(projectDir, "qrels")
colQrelsDir = os.path.join(qrelsDir, collection)
colQrelsFileName = os.path.join(colQrelsDir, "qrels.csv")
print("col Qrels File Name =", colQrelsFileName)
runsFileName = os.path.join(projectDir, "runs", method, collection, knowledgGraph, "indriRunQuery.runs") 
print("runsFileName =", runsFileName)
evalsFileName = os.path.join(projectDir, "evals", method, collection, knowledgGraph, "indriRunQuery.evals") 
print("evalsFileName =", evalsFileName)
countsResultsFile = os.path.join(projectDir,"occuranceCount","results",collection+".txt")
print("countsResultsFile =", countsResultsFile)
conceptnet5RelAllFilename ="/scratch/saeid/data/conceptnet5_simp.csv"
print("conceptnet5RelAllFilename =", conceptnet5RelAllFilename)
cfgTmpOutFileName = cfgOutFileName + ".tmp"
print("cfgTmpOutFileName =", cfgTmpOutFileName)
docIdNameMapFileName = os.path.join(projectDir, "occuranceCount", "results", collection + "_docIdNameMap.txt")
print("docIdNameMapFileName =", docIdNameMapFileName)
#expansionCount = 25

cfg infile Name = /home/fj9124/projects/ir/seq_kb_ir/configs/trec7n8/queryTrec7n8
cfg outfile Name = /home/fj9124/projects/ir/seq_kb_ir/configs/trec7n8/lr/cnet/indriRunQuery.cfg
colIndexDir = /scratch/index/indri_5_7/trec7n8
knowledgGraph index Dir = /scratch/index/indri_5_7/trec7n8
col Qrels File Name = /home/fj9124/projects/ir/seq_kb_ir/qrels/trec7n8/qrels.csv
runsFileName = /home/fj9124/projects/ir/seq_kb_ir/runs/lr/trec7n8/cnet/indriRunQuery.runs
evalsFileName = /home/fj9124/projects/ir/seq_kb_ir/evals/lr/trec7n8/cnet/indriRunQuery.evals
countsResultsFile = /home/fj9124/projects/ir/seq_kb_ir/occuranceCount/results/trec7n8.txt
conceptnet5RelAllFilename = /scratch/saeid/data/conceptnet5_simp.csv
cfgTmpOutFileName = /home/fj9124/projects/ir/seq_kb_ir/configs/trec7n8/lr/cnet/indriRunQuery.cfg.tmp
docIdNameMapFileName = /home/fj9124/projects/ir/seq_kb_ir/occuranceCount/results/trec7n8_docIdNameMap.txt


In [94]:
with open(docIdNameMapFileName, 'r') as f:
    reader = list(csv.reader(f, delimiter = ' '))
    print(reader[0])
    docIdNameMap = {i[1]:i[0] for i in reader}
    docNameIdMap = {i[0]:i[1] for i in reader}
for i, (k, v) in enumerate(docIdNameMap.iteritems()):
    print (k, '\t', v)
    if (i>20):
        break

['1', 'FR940104-1-00001']
LA082389-0020 	 441223
FBIS3-32596 	 246524
LA011890-0198 	 465144
LA011890-0199 	 465145
LA011890-0196 	 465142
LA011890-0197 	 465143
LA011890-0194 	 465140
FT932-3799 	 88713
LA011890-0192 	 465138
LA011890-0193 	 465139
LA011890-0190 	 465136
LA011890-0191 	 465137
FT932-3796 	 88710
FT924-2868 	 187695
FT924-2869 	 187696
FT932-3797 	 88711
FT924-2864 	 187691
FT924-2865 	 187692
FT924-2866 	 187693
FT924-2867 	 187694
FT924-2860 	 187687
FT924-2861 	 187688


In [3]:
count_history = dict()
with open(countsResultsFile, 'r') as f:
    reader = csv.reader(f, delimiter = "\t")
    count_history = {k:int(float(v)) for k,v in list(reader)}
print("size of count_history =", len(count_history))
for i, (k, v) in enumerate(count_history.iteritems()):
    print (k, '\t', v)
    if (i>20):
        break

size of count_history = 81446
#uw(#4( storm ) #4( be appear in sky )) 	 0
#uw(#4( work on sundai )) 	 137
#uw(#4( calm time )) 	 23
#uw(#4( kill ) #4( prai )) 	 394
#uw(#4( bank ) #4( teller offic )) 	 5
#uw(#4( water ) #4( hydrogen )) 	 347
#uw(#4( drug ) #4( take )) 	 8847
#uw(#4( space ) #4( mostli empti )) 	 3
#uw(#4( chang ) #4( edg )) 	 3803
#uw(#4( happin )) 	 0
#uw(#4( trade ) #4( countri )) 	 45838
#uw(#4( game ) #4( leisur activ )) 	 23
#uw(#4( round in shape )) 	 4
#uw(#4( good ) #4( other person )) 	 411
#uw(#4( smoke ) #4( fire )) 	 1842
#uw(#4( spice up salad )) 	 1
#uw(#4( be wound )) 	 410
#uw(#4( good ) #4( hear nice new )) 	 0
#uw(#4( teacher ) #4( teach student of class )) 	 0
#uw(#4( deliveri )) 	 11268
#uw(#4( bank ) #4( teller monei )) 	 23
#uw(#4( swap share )) 	 106


In [4]:
conceptnet5RelAll = defaultdict(dict)
conceptnet5RelAllInv = defaultdict(dict)
with open(conceptnet5RelAllFilename, 'r') as f:
    reader = csv.reader(f, delimiter = ",")
    for line in list(reader):
        if line[0].strip() != "" and line[1].strip() != "" and line[2].strip() != "":
            conceptnet5RelAll[line[1].strip()][line[2].strip()] = line[0].strip()
            conceptnet5RelAllInv[line[2].strip()][line[1].strip()] = line[0].strip()
print("size of conceptnet5RelAll =", len(conceptnet5RelAll))
for i, (k1, k2v) in enumerate(conceptnet5RelAll.iteritems()):
    for j, (k2, v) in enumerate(k2v.iteritems()):
        if (i>3 or j>3):
            break
        print (k1, '\t', k2, '\t', v)
        

size of conceptnet5RelAll = 3207471
tripolitan 	 tripoline 	 Synonym
tripolitan 	 tripoli 	 RelatedTo
age of nemesis 	 band 	 IsA
age of nemesis 	 organisation 	 InstanceOf
age of nemesis 	 progressive rock 	 dbpedia/genre
age of nemesis 	 progressive metal 	 dbpedia/genre
joseph john annabring 	 person 	 InstanceOf
british rail class 438 	 mean of transportation 	 InstanceOf
british rail class 438 	 train 	 InstanceOf


In [5]:
def splitStemText(text):
    stemmedWords = []
    text = re.sub('/|-|\"|_',' ', text) # replace - and slash with space
    if method in {"assocRestful", "lr"}:
        for text_ in text.split(' '):
            text = text_.replace(" ", "_")
            url = "http://conceptnet5.media.mit.edu/data/5.4/uri?language=en&text=" + text
            response = urllib2.urlopen(url)
            data = response.read()
            data_j = json.loads(data)
            text = os.path.basename(data_j[u'uri'])
            url = "http://conceptnet5.media.mit.edu/data/5.4/uri?language=en&text=" + text
            response = urllib2.urlopen(url)
            data = response.read()
            data_j = json.loads(data)
            #print("data_j =", data_j)
            #print("data_j[u'uri'] =", data_j[u'uri'])
            stemmedWords += [str(os.path.basename(data_j[u'uri']))]
        #print("stemmedWords =", stemmedWords)
        return (stemmedWords)
    else:
        words = text.split()
        for w in words:
            w = re.sub('\(|\)|\'s|,','', w) # remove paranthesis, apostrophe s, comma
            w = re.sub('\'','', w) # remove apostrophe
            cmd = "dumpindex " + knowledgGraphIndexDir + " t " + w + " | head -n1"
            #print("cmd =", cmd)
            stemmedWords.append(subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read().split()[1])
        return (stemmedWords)

print ("testing:", splitStemText("going reading"))

testing: ['go', 'read']


In [6]:
def analyseQueries():
    #    #print("intCoeff =", intCoeff)
    with open(cfgInFileName, 'r') as inFile:
        reader = inFile.read()
        soup = BeautifulSoup(reader, 'lxml')
        if collection in {"aquaint", "gov"}:
            tags = soup.find_all(['doc'])
        elif collection in {"robust", "trec7n8"}:
            tags = soup.find_all(['top'])
            #print(tags)
        
        origWordsAll = dict()
        for tag in tags:
            
            if collection in {"aquaint", "gov"}:
                docno = (tag.find('docno').string).strip()
            elif collection in {"robust", "trec7n8"}:
                result = re.search('<num> Number: (.*)\n', str(tag))                
                docno = result.group(1).strip()
            
            if collection in {"aquaint", "gov"}:
                origWords = splitStemText(tag.find('text').string.strip())
            elif collection in {"robust", "trec7n8"}:
                result = re.search('<title>(.*)\n', str(tag))                
                origWords = splitStemText(result.group(1).strip())
            #print("origWords =", origWords)
            origWordsAll[docno] = origWords
            
    return (origWordsAll)

In [ ]:
def countExpr_get_2(count_history, w, v):
    if w.translate(None, string.punctuation).strip() == "" or v.translate(None, string.punctuation).strip() == "":
        return 0;
    return count_history["#uw(#4( " + w.translate(None, string.punctuation) + " ) #4( " + v.translate(None, string.punctuation) + " ))"]

In [96]:
def documentLength(documentName):
    documentLength_ = subprocess.Popen(' '.join(["dumpindex", colIndexDir, "dv", docIdNameMap[documentName], "|", "awk", "\'END{print $1}\'"]), shell=True, stdout=subprocess.PIPE).stdout.read()
    return int(documentLength_)

print("docIdNameMap[\"FT944-15973\"] = ", docIdNameMap["FT944-15973"])
print("documentLength(\"FT944-15973\") =", documentLength("FT944-15973"))

def weightRelConcept(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff, count_history, featureWeights):
    
    features = dict()
    
    genOneSimpQuery(docno, origWords_, relW, intCoeff0, intCoeff1, dirCoeff, cfgTmpOutFileName)
    
    indriRun = subprocess.Popen("IndriRunQuery " + cfgTmpOutFileName, shell=True, stdout=subprocess.PIPE).stdout.read()
    indriRun = [i.split(' ') for i in indriRun.split('\n')]
    indriRun = [i for i in indriRun if len(i)>1] # remove empty arrays
    topTDocScores_d = {i[2]:np.float(i[4]) for i in indriRun}
    
    #scores = [ np.float(i[4]) for i in indriRun]
    topTDocs = [i[2] for i in indriRun]
    
    topTDocScore = np.sum([ np.float(i[4]) for i in indriRun])
    features["topTDocScore"] = topTDocScore
    
    relWDocuments = subprocess.Popen(' '.join(["dumpindex", colIndexDir, "e", "#od4(" + relW + ")"]), shell=True, stdout=subprocess.PIPE).stdout.read()
    relWDocuments = [i.split(' ') for i in relWDocuments.split('\n')]
    relWDocuments = [[docNameIdMap[i[0]], i[1], i[2], i[3]] for j, i in enumerate(relWDocuments) if j>0] # remove empty arrays
    #print("relWDocuments =", relWDocuments)
    
    relWTopDocuments = [i for i in relWDocuments if i[0] in topTDocs]
    #print("relWTopDocuments =", relWTopDocuments)
    #numerator = np.sum([int(i[1]) for i in relWTopDocuments]) # term count in the collection
    numerator = len(relWTopDocuments) # term count in the collection
    denumerator = np.sum([documentLength(i) for i in topTDocs])
    topTermFrac = numerator / np.float(denumerator)
    #print("numerator, denumerator, topTermFrac =", numerator, denumerator, topTermFrac)
    features["topTermFrac"] = topTermFrac
    
    numCanDocs = len(set([i[0] for i in relWTopDocuments]))
    features["numCanDocs"] = numCanDocs

    avgCDocScore = np.sum([ topTDocScores_d[i[0]] for i in relWTopDocuments])/np.float(numCanDocs)
    features["avgCDocScore"] = avgCDocScore
    
    maxCDocScore = np.max([ topTDocScores_d[i[0]] for i in relWTopDocuments])
    features["maxCDocScore"] = maxCDocScore
    
    conIdf = len(relWDocuments)
    features["conIdf"] = conIdf
    
    coocurDocuments = dict()
    coocurDocumentsTop = dict()
    for ow in origWords_:
        coocurDocuments_ = subprocess.Popen(' '.join(["dumpindex", colIndexDir, "e", "#uw(#4( " + ow + " ) #4( " + relW + " ))"]), shell=True, stdout=subprocess.PIPE).stdout.read()
        coocurDocuments_ = [i.split(' ') for i in coocurDocuments_.split('\n')]
        coocurDocuments[ow] = [[docNameIdMap[i[0]], i[1], i[2], i[3]] for j, i in enumerate(coocurDocuments_) if j>0] # remove empty arrays
        coocurDocumentsTop[ow] = [[i[0], i[1], i[2], i[3]] for i in coocurDocuments[ow] if i[0] in topTDocs ] # remove empty arrays

        
    avgColCor = np.sum([len(i) for i in coocurDocuments.values()])/np.float(len(coocurDocuments)) 
    features["avgColCor"] = avgColCor

    maxColCor = np.max([len(i) for i in coocurDocuments.values()])
    features["maxColCor"] = maxColCor
    
    avgTopColCor = np.sum([len(i) for i in coocurDocumentsTop.values()])/np.float(len(coocurDocumentsTop)) 
    features["avgTopColCor"] = avgTopColCor

    maxTopColCor = np.max([len(i) for i in coocurDocumentsTop.values()])
    features["maxTopColCor"] = maxTopColCor

    for j1, ow1 in enumerate(origWords_):
        for j2, ow2 in enumerate(origWords_):
            if j1>j2:
                coocurDocuments_ = subprocess.Popen(' '.join(["dumpindex", colIndexDir, "e", "#uw(#4( " + ow1 + " ) #4( " + ow2 + " ) #4( " + relW + " ))"]), shell=True, stdout=subprocess.PIPE).stdout.read()
                coocurDocuments_ = [i.split(' ') for i in coocurDocuments_.split('\n')]
                coocurDocuments[ow1+ow2] = [[docNameIdMap[i[0]], i[1], i[2], i[3]] for j, i in enumerate(coocurDocuments_) if j>0] # remove empty arrays
                coocurDocumentsTop[ow1+ow2] = [[i[0], i[1], i[2], i[3]] for i in coocurDocuments[ow1+ow2] if i[0] in topTDocs ] # remove empty arrays

    avgColPCor = np.sum([len(i) for i in coocurDocuments.values()])/np.float(len(coocurDocuments)) 
    features["avgColPCor"] = avgColPCor

    maxColPCor = np.max([len(i) for i in coocurDocuments.values()])
    features["maxColPCor"] = maxColPCor
    
    avgTopColPCor = np.sum([len(i) for i in coocurDocumentsTop.values()])/np.float(len(coocurDocumentsTop)) 
    features["avgTopColPCor"] = avgTopColPCor

    maxTopColPCor = np.max([len(i) for i in coocurDocumentsTop.values()])
    features["maxTopColPCor"] = maxTopColPCor
    
    relWg = 0
    for feature, weight in featureWeights.iteritems():
        relWg += features[feature] * weight
        
    return relWg

docIdNameMap["FT944-15973"] =  6961
documentLength("FT944-15973") = 287


In [88]:
def genOneSimpQuery(docno, origWords_, relW, intCoeff0, intCoeff1, dirCoeff, cfgTmpOutFileName):
    #    #print("intCoeff =", intCoeff)
    soupNew = BeautifulSoup("", 'lxml')
    parameters_tag = soupNew.new_tag("parameters")
    index_tag = soupNew.new_tag("index")
    index_tag.string = colIndexDir
    parameters_tag.append(index_tag)
        #print(tags)

    doc_tag = soupNew.new_tag("query")

    docno_tag = soupNew.new_tag("number")
    docno_tag.string = docno
    doc_tag.append(docno_tag)

    text_tag = soupNew.new_tag("text")
    text_tag.string = "#weight(\n" 

    text_tag.string += str(intCoeff0) + " #combine(" 
    for ow in set(origWords_):
        ow = regex.sub(' ', ow)
        text_tag.string += ow + " "
    text_tag.string += ")\n"

    if len(relW)>0: 
        relText_string1 = relW
        text_tag.string += str(intCoeff1) + " #combine(" 
        #print("relText_string1 =", relText_string1)
        text_tag.string += relText_string1.encode('utf-8')
        text_tag.string += ")\n"

    text_tag.string += ") "

    doc_tag.append(text_tag)

    parameters_tag.append(doc_tag)
    #print(doc_tag)

    rule_tag = soupNew.new_tag("rule")
    rule_tag.string = "method:dir,mu:" + str(dirCoeff)
    #rule_tag.string = "method:two"
    parameters_tag.append(rule_tag)

    #intCoeff_tag = soupNew.new_tag("intCoeff")
    #intCoeff_tag.string = "0.8"
    #parameters_tag.append(intCoeff_tag)

    threads_tag = soupNew.new_tag("threads")
    threads_tag.string = "32"
    parameters_tag.append(threads_tag)

    count_tag = soupNew.new_tag("count")
    count_tag.string = "10"
    parameters_tag.append(count_tag)

    trecFormat_tag = soupNew.new_tag("trecFormat")
    trecFormat_tag.string = "true"
    parameters_tag.append(trecFormat_tag)

    soupNew.append(parameters_tag)
    #print(soupNew.prettify())
    #print("outFileName =", outFileName)
    with open( cfgTmpOutFileName, 'w') as outFile:
        soupNewStr = str(soupNew)
        soupNewStr = soupNewStr.replace("</text>", "\n</text>\n").replace("query>", "query>\n").replace("<text>", "\n<text>\n").replace("</index>", "</index>\n").replace("\n<index>", "<index>")

        outFile.write(soupNewStr)

In [89]:
# keys of rWords are important
# values of rWords1 are important
regex = re.compile('[%s]' % re.escape(string.punctuation))
def getRelWords(count_history, rWords, docno, origWords, expansionCount, intCoeff0, intCoeff1, intCoeff2, dirCoeff):
    #print("origWords =", origWords)
    text_string = ""
    
        
    relWords = dict()
    #if method == "relAll":
        
    if method == "assocRestful":
        origWords_ = [w.replace(" ", "_") for w in origWords]
        url = "http://conceptnet5.media.mit.edu/data/5.4/assoc/list/en/"+','.join(origWords_)
        response = urllib2.urlopen(url)
        data = response.read()
        data_j = json.loads(data)
        relWords = {str(d[0].encode('utf-8')).replace('/c/en/', '').replace('-', ' '):np.float(d[1]) for d in data_j[u'similar'] if '/c/en/' in str(d[0].encode('utf-8'))}
    
    elif method == "lr":
        origWords_ = [w.replace(" ", "_") for w in origWords]
        #relWords_l = []
        for ow in origWords_:
            #relWords_l += list(conceptnet5RelAll[ow])
            #print(ow, "---", conceptnet5RelAll[ow])
            for relW in conceptnet5RelAll[ow].keys():
                relWg = weightRelConcept(docno, origWords_, ow, relW, intCoeff0, intCoeff1, intCoeff2, dirCoeff)
                if relW in relWords:
                    relWords[relW] += relWg
                else:
                    relWords[relW] = relWg   
    
    else:
        count_history = countExpr(count_history, origWords, rWords)
        expressions = set()
        for ow in origWords:
            rWords_ow = rWords.get(ow, [])
            expressions.add("#4( " + ow.translate(None, string.punctuation) + " )")
            if ow.strip() == "":
                continue
            if simMeasure == "mi":
                N_w = countExpr_get_1(count_history, ow)
            for rw_wv in rWords_ow:
                (rw_w, rw_v) = rw_wv.items()[0]
                if rw_w.strip() == "":
                    continue           
                if simMeasure == "mi":
                    N_v = countExpr_get_1(count_history, rw_w)
                    N_wv = countExpr_get_2(count_history, ow, rw_w)
                    rw_v_ = mi_(N_w, N_v, N_wv)
                elif simMeasure == "cnet":
                    rw_v_ = rw_v
                relWords[rw_w] = relWords.get(rw_w, 0) + rw_v_
    
    relWords_sorted = sorted(relWords.items(), key=operator.itemgetter(1), reverse=True)
    #print ("origWords =", origWords)
    #print ("relWords_sorted =", relWords_sorted)
    #counter = 0
    relWords_sel = []
    for counter, (rw_w, rw_v) in enumerate(relWords_sorted):
        if (counter >= expansionCount):
            break
        if  all(c in string.printable for c in rw_w):
            if rw_w not in origWords:
                rw_w = regex.sub(' ', rw_w)
                text_string += rw_w + " "
                relWords_sel += [rw_w]
        #counter += 1
    #print("counter =", counter)
    return(count_history, text_string, relWords_sel)

In [90]:
def getAllRelWords(count_history, rWords, origWordsAll, expansionCount1, N1, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff):
    relText_stringAll = []
    relText_string = dict()
    relWords_sel = dict()
    for docno, origWords in origWordsAll.iteritems():
        count_history, relText_string_, relWords_sel_ = getRelWords(count_history, rWords, docno, origWords, expansionCount1, intCoeff0, intCoeff1, intCoeff2, dirCoeff)
        relText_string[docno] = relText_string_
        relWords_sel[docno] = relWords_sel_[0:N1]
        #print(docno, "---", origWords, "---", relWords_sel_)
    relText_stringAll += [relText_string]
    
    relText_string = dict()
    relWords_sel = dict()
    for docno, origWords_new in relWords_sel.iteritems():
        count_history, relText_string_, relWords_sel_ = getRelWords(count_history, rWords, docno, origWords_new, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff)
        relText_string[docno] = relText_string_
        relWords_sel[docno] = relWords_sel_
    relText_stringAll += [relText_string]
    
    #print("relText_stringAll =", relText_stringAll)
    return (count_history, relText_stringAll)

In [91]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
def genQueries(origWordsAll, relText_stringAll, intCoeff0, intCoeff1, intCoeff2, dirCoeff):
    #    #print("intCoeff =", intCoeff)
    with open(cfgInFileName, 'r') as inFile:
        reader = inFile.read()
        soupNew = BeautifulSoup("", 'lxml')
        parameters_tag = soupNew.new_tag("parameters")
        index_tag = soupNew.new_tag("index")
        index_tag.string = colIndexDir
        parameters_tag.append(index_tag)
            #print(tags)

        for docno, origWords in origWordsAll.iteritems():
            doc_tag = soupNew.new_tag("query")

            docno_tag = soupNew.new_tag("number")
            docno_tag.string = docno
            doc_tag.append(docno_tag)

            text_tag = soupNew.new_tag("text")
            text_tag.string = "#weight(\n" 
                    
            text_tag.string += str(intCoeff0) + " #combine(" 
            for ow in set(origWords):
                ow = regex.sub(' ', ow)
                text_tag.string += ow + " "
            text_tag.string += ")\n"

            if len(relText_stringAll[0])>0:
                if len(relText_stringAll[0][docno]) > 3: 
                    relText_string1 = relText_stringAll[0][docno]
                    text_tag.string += str(intCoeff1) + " #combine(" 
                    #print("relText_string1 =", relText_string1)
                    text_tag.string += relText_string1.encode('utf-8')
                    text_tag.string += ")\n"
            
            #print(len(relText_stringAll[1]))
            if len(relText_stringAll[1])>0:
                if len(relText_stringAll[1][docno]) > 3: 
                    relText_string2 = relText_stringAll[1][docno]
                    text_tag.string += str(intCoeff2) + " #combine(" 
                    #print("relText_string2 =", relText_string2)
                    text_tag.string += relText_string2.encode('utf-8')
                    text_tag.string += ")\n"
            
            #print("origWords =", origWords)
            #print("relWords_sel1 =", relWords_sel1)
            #print("relWords_sel2 =", relWords_sel2)
            
            
            text_tag.string += ") "

            doc_tag.append(text_tag)

            parameters_tag.append(doc_tag)
            #print(doc_tag)

        rule_tag = soupNew.new_tag("rule")
        rule_tag.string = "method:dir,mu:" + str(dirCoeff)
        #rule_tag.string = "method:two"
        parameters_tag.append(rule_tag)

        #intCoeff_tag = soupNew.new_tag("intCoeff")
        #intCoeff_tag.string = "0.8"
        #parameters_tag.append(intCoeff_tag)

        threads_tag = soupNew.new_tag("threads")
        threads_tag.string = "32"
        parameters_tag.append(threads_tag)

        count_tag = soupNew.new_tag("count")
        count_tag.string = "1000"
        parameters_tag.append(count_tag)

        trecFormat_tag = soupNew.new_tag("trecFormat")
        trecFormat_tag.string = "true"
        parameters_tag.append(trecFormat_tag)

        soupNew.append(parameters_tag)
        #print(soupNew.prettify())
    #print("outFileName =", outFileName)
    with open( cfgOutFileName, 'w') as outFile:
        soupNewStr = str(soupNew)
        soupNewStr = soupNewStr.replace("</text>", "\n</text>\n").replace("query>", "query>\n").replace("<text>", "\n<text>\n").replace("</index>", "</index>\n").replace("\n<index>", "<index>")

        outFile.write(soupNewStr)


In [92]:
def precisionCompute(count_history, intCoeffs0, intCoeffs1, intCoeffs2, expansionCounts1, expansionCounts2, dirCoeffs, N1s):
    mapPrecs = dict()
    for N1 in N1s:
        for intCoeff0 in intCoeffs0:
            for intCoeff1 in intCoeffs1:
                for intCoeff2 in intCoeffs2:
                    for expansionCount1 in expansionCounts1:
                        for expansionCount2 in expansionCounts2:
                            for dirCoeff in dirCoeffs:
                                origWordsAll = analyseQueries()
                                count_history, relText_stringAll = getAllRelWords(count_history, rWords, origWordsAll, expansionCount1, N1, expansionCount2, intCoeff0, intCoeff1, intCoeff2, dirCoeff)
                                print("len(relText_stringAll[1]) =", len(relText_stringAll[1]))
                                genQueries(origWordsAll, relText_stringAll, intCoeff0, intCoeff1, intCoeff2, dirCoeff)
                                subprocess.Popen("IndriRunQuery " + cfgOutFileName + " > " + runsFileName, shell=True, stdout=subprocess.PIPE).stdout.read()
                                cmd = "trec_eval -q " + colQrelsFileName + " " + runsFileName + " > " + evalsFileName
                                subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
                                cmd = "cat " + evalsFileName + " | grep map | grep all | grep -v gm | awk '{print $3}' "
                                #print ("cmd = ", cmd)
                                mapPrec = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE).stdout.read()
                                print("intCoeff0, intCoeff1, intCoeff2, expansionCount1, expansionCount2, dirCoeff, map precision, N1 =", intCoeff0, intCoeff1, intCoeff2, expansionCount1, expansionCount2, dirCoeff, N1, mapPrec)
                                #mapPrecs[intCoeff] = mapPrec
    return count_history

In [97]:
intCoeffs0 = [ 0.7 ]
intCoeffs1 = [ 0.3 ]
#intCoeffs2 = [ 0.18 ]
intCoeffs2 = [ 0.5 ]
#intCoeffs1 = np.arange(0.1, 1, 0.1)
expansionCounts1 = [ 1000 ]
#expansionCounts2 = [ 145 ]
expansionCounts2 = [ 0 ]
#expansionCounts2 = range(15, 220, 5)
dirCoeffs = [ 1600 ]
#dirCoeffs = range(200, 4000, 200)
N1s = [0]
#N1s = range(1, 30, 1)
#mapPrecs = 
rWords = dict()
count_history = precisionCompute(count_history, intCoeffs0, intCoeffs1, intCoeffs2, expansionCounts1, expansionCounts2, dirCoeffs, N1s)


relWTopDocuments = [['FT941-5453', '1', '411'], ['FT921-6003', '2', '1252'], ['FT933-9176', '3', '826'], ['FBIS4-67534', '1', '400'], ['FBIS3-60235', '1', '237'], ['LA112890-0007', '1', '1143'], ['LA102090-0048', '5', '1879'], ['LA100189-0051', '3', '686'], ['LA021190-0107', '1', '710']]
numerator, denumerator, topTermFrac = 18 8747 0.00205784840517
relWTopDocuments = []
numerator, denumerator, topTermFrac = 0.0 11361 0.0
relWTopDocuments = [['FT941-15416', '4', '409'], ['FT941-5453', '3', '411'], ['FT921-6003', '7', '1252'], ['FT933-7330', '3', '1540'], ['FT933-9176', '4', '826'], ['FBIS3-41533', '8', '476'], ['FBIS4-67534', '2', '400'], ['LA112890-0007', '2', '1143'], ['LA021190-0107', '2', '710'], ['LA050889-0063', '11', '1213']]
numerator, denumerator, topTermFrac = 46 8370 0.00549581839904
relWTopDocuments = [['LA082189-0001', '1', '2833'], ['LA082390-0149', '1', '431'], ['LA122790-0033', '1', '900'], ['LA111190-0042', '1', '1928'], ['LA101890-0141', '1', '725']]


KeyboardInterrupt: 